In [1]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device")
else:
    device = torch.device("cpu")
    print("Using CPU device")

Using MPS device


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from transformers import pipeline
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
import transformers

import torch

import textwrap

import bert_score

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


In [ ]:
generator = pipeline('text-generation', model='gpt2-xl', device="mps")